# Notebook for analyzing mammarenavirus GPC sequences and designing a LASV GP library

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

from plotnine import *
from Bio import SeqIO, AlignIO, Phylo

from Bio.Align import MultipleSeqAlignment

In [ ]:
CBP = ('#999999', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7')
theme_set(theme_seaborn(style='darkgrid', context='talk', font_scale=1))

### Set up percent identiity functions

In [ ]:
def percent_ids(s1, s2, id2):
    assert len(s1) == len(s2), 'sequences of unequal length'
    length = len(s1)
    num_gaps = s1.count('-')
    num_aas = length - num_gaps
    difs_aa = 0
    difs_gap1 = 0
    difs_gap2 = 0
    id_gaps = 0
    for i in range(length):
        if s1[i] == '-':
            if s2[i] != '-':
                difs_gap1 += 1
            else:
                id_gaps += 1
                
        elif s2[i] == '-':
            difs_gap2 += 1
            
        elif s1[i] != s2[i]:
            difs_aa += 1
        
        else:
            assert s1[i] == s2[i], f"ID determination error: s1 = {s1[i]}, s2={s2[i]}"
   
    pid_dict = {'seq_id': [id2],
                'pid_aa': [(num_aas-difs_aa)/num_aas], 
                'pid_gaps': [(id_gaps)/num_gaps],
                'p_aa1_gap2': [(difs_gap2)/num_aas],
                'p_gap1_aa2': [(difs_gap1)/num_gaps]}
    
    return(pd.DataFrame(pid_dict))

In [ ]:
percent_ids('-G-AT', '-CG-T', 'test')

## Clean up downloaded sequences

First look at all sequences and checkout length distribution.

I started with the "Mammarenavirus GPC" search on GenBank, but also did a "Mammarenavirus glycoprotein" GenBank search and a couple searches of the [VIPR database](https://www.viprbrc.org/brc/search_landing.spg?decorator=arena). I will look at length distributions for all of those. 

In [ ]:
def df_len_plot(file):
    with open(file) as fasta_file:
        ids = []
        seqs = []
        lengths = []
        descriptions = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):
            ids.append(seq_record.id)
            lengths.append(len(seq_record.seq))
            seqs.append(str(seq_record.seq))
            descriptions.append(seq_record.description)

    seqs_df = pd.DataFrame(dict(ID=ids, Length=lengths, Description=descriptions, Seq=seqs))
    
    print(f"Sequence count: {len(seqs_df)}")
    display(seqs_df.head())
    
    lengths_plot = (ggplot(seqs_df, aes(x='Length')) +
                    geom_histogram(binwidth=10) 
                   )

    _ = lengths_plot.draw()
    
    return(seqs_df)

In [ ]:
gb_mammarenavirus_gpc = df_len_plot('./seq_downloads/mammarenavirus_gpcs_all.fasta')

In [ ]:
gb_mammarenavirus_glycoprotein = df_len_plot('./seq_downloads/mammarenavirus_glycoprotein_all.fasta')

In [ ]:
vipr_mammarenavirus = df_len_plot('./seq_downloads/mammarenavirus_gp_viprbrc.fasta')

In [ ]:
vipr_arenaviridae = df_len_plot('./seq_downloads/arenaviridae_gp_viprbrc.fasta')

Most sequences from the GenBank "GPC" search look like full-length GPC, right around 500 amino acids in length. There are clearly some mislablled sequennces in the "glycoprotein" search, but even still there are more full-length sequences there. 

However, most of the VIPR sequences seem to be partial sequences. Looking at the VIPR sequences more, it looks like most of the human LASV sequences were split into GP1 and GP2 sequences in the database. As such, I will ignore the VIPR sequences and just use the GenBank sequences. 

As far as I can tell, all the VIPR sequences are also indexed in genbank, so this should still be most of the available mammarenavirus sequences. 

Note: I am starting with a mammarenavirus alignment. This includes New World and Old World mammarenaviruses, which use two different receptors (alpha-dystroglycan for Old World and transferin receptor for New World). 

# Set seqs_df

Set `seqs_df` to be the collection of mammarenavirus sequences downloaded from genbank with the "glycoprotein" search term.

In [ ]:
seqs_df = gb_mammarenavirus_glycoprotein

## Filter sequences by length

Based on the above histogram, it seems reasonable to include sequences >405 amino acids and <550 amino acids in length

In [ ]:
seqs_df = seqs_df[(seqs_df['Length'] < 550) & (seqs_df['Length'] > 405)]
print(len(seqs_df))
display(seqs_df.head())

## Filter out sequences with > 10 `X`s

In [ ]:
seqs_df = seqs_df[(seqs_df['Seq'].str.count('X')<=10)]
print(len(seqs_df))
display(seqs_df.head())

## Subset on LASV Seqs for initial alignment

In [ ]:
lasv_seqs_df = seqs_df[(seqs_df['Description'].str.contains('Lassa'))]
print(len(lasv_seqs_df))
display(lasv_seqs_df.head())

In [ ]:
jos_id = 'NP_694870.1'
jos_seq_df = seqs_df[(seqs_df['Description'].str.contains(jos_id))]
display(jos_seq_df)

In [ ]:
g1959_id = 'AIT17216.1'
g1959_seq_df = seqs_df[(seqs_df['Description'].str.contains(g1959_id))]
display(g1959_seq_df)

## Output filtered sequences to `fasta` files for alignment

In [ ]:
with open('./gpcs_filtered.fasta', 'w') as outfile:
    for idx in range(len(seqs_df)):
        outfile.write(f">{seqs_df.iloc[idx]['ID']}\n")
        outfile.write(f"{seqs_df.iloc[idx]['Seq']}\n")
        
with open('./lasv_gpcs.fasta', 'w') as outfile:
    for idx in range(len(lasv_seqs_df)):
        outfile.write(f">{lasv_seqs_df.iloc[idx]['ID']}\n")
        outfile.write(f"{lasv_seqs_df.iloc[idx]['Seq']}\n")

## Use `mafft` to align filtered GPC sequences

In [ ]:
gpc_align_outfile = './gpc_filtered_alignment.fasta'
lasv_align_outfile = './lasv_alignment.fasta'

In [ ]:
if not os.path.isfile(gpc_align_outfile):
    ! mafft --retree 2 --maxiterate 1000 --quiet 'gpcs_filtered.fasta' > './gpc_filtered_alignment.fasta'
    print('Aligned GPC sequences with MAFFT (MAFFT output silenced).')
else:
    print('GPC alignment already exists. Using existing alignment.')



In [ ]:
if not os.path.isfile(lasv_align_outfile):
    ! mafft --retree 2 --maxiterate 1000 --quiet 'lasv_gpcs.fasta' > './lasv_alignment.fasta'
    print('Aligned LASV sequences with MAFFT (MAFFT output silenced).')
else:
    print('LASV alignment already exists. Using existing alignment.')

# Look at LASV GP Alignment

In [ ]:
lasv_align = AlignIO.read(lasv_align_outfile, "fasta")
print(f"{len(lasv_align)} sequences")

### Extract LASV Josiah GP sequence as reference

In [ ]:
for seq in lasv_align:
    if seq.description == jos_id:
        jos_align = seq

print(jos_align)
print()
print(jos_align.seq)

## Create df of percent IDs for LASV GP seqs

In [ ]:
lasv_pids_df = pd.DataFrame()
for seq in lasv_align:
    lasv_pids_df = pd.concat([lasv_pids_df, percent_ids(jos_align.seq, seq.seq, seq.id)]).reset_index(drop=True)
    
display(lasv_pids_df.sort_values('pid_aa', ascending=True))

In [ ]:
pids_plot = (ggplot(lasv_pids_df, aes(x='pid_aa')) +
                    geom_histogram(binwidth=0.005) +
                    theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5)) +
                    scale_x_continuous(limits=[0.88, 1.0], breaks=[0.88, 0.9, 0.92, 0.94, 0.96, 0.98, 1.0])
               )

_ = pids_plot.draw()

## Make Phylogenetic Tree

In [ ]:
lasv_tree_outfile = './lasv.treefile'

# use Pinneo strain as outgroup, as typically done.

if not os.path.isfile(lasv_tree_outfile):
    ! iqtree -s lasv_alignment.fasta -pre lasv -nt 4 -m LG+F+G -o 'AIT17836.1' -quiet
else:
    print('LASV tree already constructed. Using existing tree.')

In [ ]:
lasv_lineages = {'I: Pinneo': ['AIT17836.1'], 'II: Nigeria (S)': ['ADU56610.1', 'AAF86703.1'], 'III: Nigeria (C)': ['ADU56618.1', 'CAA36645.1'],
                 'IV: Sierra Leone (Josiah)': [jos_id], 'IV: Sierra Leone': [g1959_id], 'IV: Liberia': ['AAT49008.1'], 'IV: Guinea': ['AAT49000.1'],
                 'Guinea (mouse)': ['ADI39451.1'], 'SL (mouse)': ['AIT17840.1'], 'Nigeria': ['AIT17646.1', 'AIT17556.1', 'AIT17540.1'],
                 'Nigeria (CSF)': ['AAL13212.1'], 'Benin (mouse)': ['QCF45564.1'],
                 'V: Mali, Ivory Coast': ['AHC95553.1'], 'VI: Nigeria (Hylomyscus)': ['ANH09740.1'], 'VII: Togo': ['SCA79105.1']}

lasv_tree_labels = [item for sublist in lasv_lineages.values() for item in sublist]
print(lasv_tree_labels)

In [ ]:
def get_key(val, my_dict): 
    for key, value in my_dict.items():
        if val in value: 
            return(key)

In [ ]:
tree = Phylo.read(lasv_tree_outfile, "newick")
# tree.root_at_midpoint()
# tree.ladderize()  # Flip branches so deeper clades are displayed at top

treedir = './'

treefigfile = os.path.join(treedir, 'lasv_tree_plot.pdf')

Phylo.draw(tree, label_func=lambda x: get_key(str(x), lasv_lineages) if str(x) in lasv_tree_labels else '', 
           label_colors=dict([(lineage, 'red') for lineage in lasv_lineages.keys()]),
           do_show=False)


treefig = plt.gcf()
treefig.set_size_inches(7, 24)
ax = treefig.axes[0]
ax.axis('off')

# add scale bar
(x0, x1, y0, y1) = plt.axis()
xstart = x0 + 0.3 * (x1 - x0)
barlen = 0.1
yline = y0 - 0.05 * (y1 - y0)
ytext = yline - 0.01 * (y1 - y0)
ax.set_ylim(ytext, y1)
plt.plot([xstart, xstart + barlen], [yline, yline], color='black', 
        linestyle='-', linewidth=2)
plt.text(xstart + barlen / 2., ytext, 
        '{0:.1} amino acid substitutions / site'.format(barlen),
        horizontalalignment='center',
        verticalalignment='top',
        color='black',
        fontsize=17)

treefig.tight_layout()
treefig.savefig(treefigfile)


### Find gaps in Josiah GP

In [ ]:
gaps = []
for i in range(len(jos_align.seq)):
    if jos_align.seq[i] == '-':
        gaps.append(i)
        
print(gaps)

Drop first 35 sites and last site of alignment so don't include gaps in Josiah alignment.

In [ ]:
lasv_align_nogaps=lasv_align[:, 35:526]
jos_align_nogaps=jos_align[35:526]
print(jos_align_nogaps.seq)

# Clean up code:

## Make character counting and plotting code functions so can run on the different alignments more easily

### Calculate counts for each character

In [ ]:
def process_alignment(refseq, alignment, mincount=1):
    site_char_counts = {}
    for i in range(len(refseq.seq)):
        char_counts = {}
        for char in set(alignment[:, i]):
            char_counts[char] = alignment[:, i].count(char)
        site_char_counts[i] = char_counts
    
    site_char_counts_df = pd.DataFrame.from_dict(site_char_counts)
        
    # transpose df so site is index, then drop cells where count less than mincount
    scc_transpose = site_char_counts_df.transpose()
    
    if mincount != 1:
        scc_transpose = scc_transpose.where(scc_transpose >= mincount)
    
    # make site column with 1-indexing
    scc_transpose['site'] = scc_transpose.index+1

    # add in info on max counts for each aa at each site and num dif aa at each site
    # first subset on df excluding gaps and xs
    scc_nogaps_nox = scc_transpose.drop(['-', 'X', 'site'], axis=1)
    # add 'max_nogap' column for max count for any given aa at site without counting gaps
    scc_transpose['max_nogap'] = scc_nogaps_nox[scc_nogaps_nox.columns].max(axis=1)
    # add 'num_dif_nogap' column for num of dif aas at each site not including xs or gaps
    scc_transpose['num_dif_nogap'] = (scc_nogaps_nox.count(axis=1)-1) # only count "extra" aas, not first
    # fill na
    scc_transpose = scc_transpose.fillna(0)
    
    display(scc_transpose.head())
    
    return scc_transpose

In [ ]:
lasv_align_df = process_alignment(jos_align_nogaps, lasv_align_nogaps)

### Make plots:

1. Max count for any one amino acid at each site
2. Number gaps at each site
3. Number different amino acids at each site

In all plots, the rough separations are colored as below:
* SSP/GP1: blue dashed line
* GP1/GP2: green dashed line
* GP2/TM: orange dashed line

In [ ]:
def make_line_plots(align_df, subset_name):
    max_count_plot = (ggplot(align_df, aes(x='site', y='max_nogap')) +
                         geom_line() +
                         geom_vline(xintercept=59.5, linetype='dashed', color='blue') +
                         geom_vline(xintercept=259.5, linetype='dashed', color='green') +
                         geom_vline(xintercept=427.5, linetype='dashed', color='orange') +
                        ggtitle(f"{subset_name}: max_count_plot")
                 )

    _ = max_count_plot.draw()
    
    num_gap_plot = (ggplot(align_df, aes(x='site', y='-')) +
                       geom_line() +
                       geom_vline(xintercept=59.5, linetype='dashed', color='blue') +
                       geom_vline(xintercept=259.5, linetype='dashed', color='green') +
                       geom_vline(xintercept=427.5, linetype='dashed', color='orange') +
                       ggtitle(f"{subset_name}: num_gap_plot")
               )

    _ = num_gap_plot.draw()
    
    num_dif_plot = (ggplot(align_df, aes(x='site', y='num_dif_nogap')) +
                       geom_line() +
                       geom_vline(xintercept=59.5, linetype='dashed', color='blue') +
                       geom_vline(xintercept=259.5, linetype='dashed', color='green') +
                       geom_vline(xintercept=427.5, linetype='dashed', color='orange') +
#                        scale_y_continuous(breaks=range(0, 12, 1)) +
                       ggtitle(f"{subset_name}: num_dif_plot")
               )

    _ = num_dif_plot.draw()

In [ ]:
make_line_plots(lasv_align_df, 'LASV')

In [ ]:
print(lasv_align[50:100, 85:105]) # indices of full alignment shifted 35 from Jos alignment due to initial gap
print(jos_align.seq[85:105])

Clearly, a lot of sequences have a gap at site 60 (note, with "auto" alignment this gap was at site 59 or 60). 

Looking at [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7232328/), having a residue at site 60 seems to be unique to Lineage IV viruses (such as Josiah and G1959 strains). Note also that my plotting indexing is 1-indexing, but the alignment indexing is 0-indexed.

Note the SSP/GP1 cleavage site is between the two `T`s at sites 57/58. 

Looking at a few of the alignments (above) it seems like this gap is real, not some sequence alignment artifact. I should dig into if LASV GPs from different lineages are known to either have or not have this gap.

In [ ]:
conserved_sites = lasv_align_df[lasv_align_df['num_dif_nogap']==0]['site']
conserved_sites_ectodomain = [x for x in conserved_sites if x <427]
print(conserved_sites_ectodomain)
print('\nNumber of conserved sites:')
print(len(conserved_sites_ectodomain))

According to the above analysis, there are 260 sites with no variability in the LASV GP alignment.

Not mutating those sites would just about halve the library size.

I will next look at the full mammarenavirus alignment to see if therer are sites with diversity in the full alignment.

# Look at full mammarenavirus GPC alignment

In [ ]:
gpc_align = AlignIO.read("gpc_filtered_alignment.fasta", "fasta")
print(f"{len(gpc_align)} sequences")

Find Josiah sequence:

In [ ]:
for seq in gpc_align:
    if seq.description == jos_id:
        jos_full_align = seq

print(jos_full_align)
print()
print(jos_full_align.seq)

print(len(jos_full_align.seq))
print(jos_full_align.seq.count('-'))
print(len(jos_full_align.seq) - jos_full_align.seq.count('-'))

## Look at percent ids in full alignment

In [ ]:
all_pids_df = pd.DataFrame()
for seq in gpc_align:
    all_pids_df = pd.concat([all_pids_df, percent_ids(jos_full_align.seq, seq.seq, seq.id)]).reset_index(drop=True)
    
display(all_pids_df.sort_values('pid_aa', ascending=True))

In [ ]:
all_pids_plot = (ggplot(all_pids_df, aes(x='pid_aa')) +
                    geom_histogram(binwidth=0.005) +
                    theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5)) +
                    scale_x_continuous(limits=[0.4, 1.0], breaks=[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1])
               )

_ = all_pids_plot.draw()

In [ ]:
low_pid_df = all_pids_df[(all_pids_df['pid_aa'] < 0.55)].sort_values('pid_aa')
display(low_pid_df.head())
low_pid_ids = low_pid_df['seq_id']

Based on GenBank searches of IDs with percent amino acid identities <55%, the group of GPs with the least identity to LASV GP Josiah seem to all be New World mammarenaviruses. This makes sense as most of those use a different receptor.

In [ ]:
display(all_pids_df[(all_pids_df['pid_aa'] >= 0.55) & (all_pids_df['pid_aa'] < 0.7)].sort_values('pid_aa').head())

The next cluster of around 60% identity is Old World mammarenavirus, mostly LCMV.

In [ ]:
display(all_pids_df[(all_pids_df['pid_aa'] >= 0.7) & (all_pids_df['pid_aa'] < 0.756)].sort_values('pid_aa').head())
display(all_pids_df[(all_pids_df['pid_aa'] >= 0.7) & (all_pids_df['pid_aa'] < 0.756)].sort_values('pid_aa').tail())

From 0.7 - 0.75 percent aa identity, are Old World mammarenaviruses, including many (mostly) from SE Asia.

In [ ]:
display(all_pids_df[(all_pids_df['pid_aa'] >= 0.756) & (all_pids_df['pid_aa'] < 0.8)].sort_values('pid_aa').head())
display(all_pids_df[(all_pids_df['pid_aa'] >= 0.756) & (all_pids_df['pid_aa'] < 0.8)].sort_values('pid_aa').tail())

From 0.75-0.8 percent aa identity are more Old World arenaviruses mainly (mostly) in Africa, largely of infecting the same/similar rodents as LASV.

# Make tree from full alignment

In [ ]:
gpc_tree_outfile = './gpc_all.treefile'

if not os.path.isfile(gpc_tree_outfile):
    ! iqtree -s gpc_filtered_alignment.fasta -pre gpc_all -nt AUTO -m LG+F+G -quiet
else:
    print('GPC tree already constructed. Using existing tree.')

In [ ]:
gpc_labels = {'LASV I': ['AIT17836.1'], 'LASV II': ['ADU56610.1', 'AAF86703.1'], 'LASV III': ['ADU56618.1', 'CAA36645.1'],
              'LASV IV': [jos_id, g1959_id, 'AAT49008.1', 'AAT49000.1'],
              'LASV Guinea (mouse)': ['ADI39451.1'], 'LASV SL (mouse)': ['AIT17840.1'], 'LASV Nigeria (CSF)': ['AAL13212.1'],
              'LASV V': ['AHC95553.1'], 'LASV VI': ['ANH09740.1'], 'LASV VII': ['SCA79105.1'], 
              'LCMV': ['AAA46256.1', 'AAA46265.1'], 'Ippy': ['YP_516230.1'], 'Oliveros': ['AAC54654.1'], 'Pichinde': ['AAA46824.1'],
              'Junin': ['AAU34180.1'], 'Tacaribe': ['NP_694849.1'], 'Machupo': ['NP_899212.1'],
              'Mopeia': ['AAC08700.1', 'ABC71134.1'], 'Mobala': ['YP_516226.1'], 'Guanarito': ['NP_899210.1'], 
              'Catarina': ['ABI97298.1'], 'Bear Canyon': ['AAN32965.1'], 'Flexal': ['AAN32961.1'],
              'Lujo': ['YP_002929490.1'], 'Latino': ['AAN32959.1'], 'Wenzhou': ['YP_009113206.1'], 'Luna': ['BAU22158.1']}

gpc_colors = {'LASV I': 'red', 'LASV II': 'red', 'LASV III': 'red', 'LASV IV': 'red', 'LASV Guinea (mouse)': 'red',
              'LASV SL (mouse)': 'red', 'LASV Nigeria (CSF)': 'red', 'LASV V': 'red', 'LASV VI': 'red', 'LASV VII': 'red', 
              'LCMV': 'orange', 'Ippy': 'orange', 'Oliveros': 'blue', 'Pichinde': 'blue',
              'Junin': 'blue', 'Tacaribe': 'blue', 'Machupo': 'blue', 'Lujo': 'orange',
              'Mopeia': 'orange', 'Mobala': 'orange', 'Guanarito': 'blue', 'Catarina': 'blue',
              'Bear Canyon': 'blue', 'Flexal': 'blue', 'Latino': 'blue', 'Wenzhou': 'orange', 'Luna': 'orange'}

gpc_tree_labels = [item for sublist in gpc_labels.values() for item in sublist]

In [ ]:
#display pids for labeled viruses and LASV
for virus in gpc_labels:
    print(virus)
    seqid_list = gpc_labels[virus]
    display(all_pids_df[(all_pids_df['seq_id'].isin(seqid_list))])

In [ ]:
tree = Phylo.read(gpc_tree_outfile, "newick")
tree.root_at_midpoint()
tree.ladderize()  # Flip branches so deeper clades are displayed at top

treedir = './'

treefigfile = os.path.join(treedir, 'gpc_tree_plot.pdf')

Phylo.draw(tree, label_func=lambda x: get_key(str(x), gpc_labels) if str(x) in gpc_tree_labels else '', 
           label_colors=gpc_colors,
           do_show=False)


treefig = plt.gcf()
treefig.set_size_inches(7, 30)
ax = treefig.axes[0]
ax.axis('off')

# add scale bar
(x0, x1, y0, y1) = plt.axis()
xstart = x0 + 0.3 * (x1 - x0)
barlen = 0.1
yline = y0 - 0.05 * (y1 - y0)
ytext = yline - 0.01 * (y1 - y0)
ax.set_ylim(ytext, y1)
plt.plot([xstart, xstart + barlen], [yline, yline], color='black', 
        linestyle='-', linewidth=2)
plt.text(xstart + barlen / 2., ytext, 
        '{0:.1} amino acid substitutions / site'.format(barlen),
        horizontalalignment='center',
        verticalalignment='top',
        color='black',
        fontsize=17)

treefig.tight_layout()
treefig.savefig(treefigfile)

### Create alignments that are just Old or New World Arenaviruses

Made these alignments, but decided to continue doing analyses with full alignment becuase: 

1. I will just use degenerate nucleotides for GP1 anyway
2. I only want to exclude _really_ conserved sites in GP2

Although, really I should probably do these analyses for both with and without the New World sequences included. 

In [ ]:
gpc_align_ow = MultipleSeqAlignment([])
gpc_align_nw = MultipleSeqAlignment([])

for i in range(len(gpc_align)):
    if gpc_align[i].id in low_pid_ids.to_list():
        gpc_align_nw.append(gpc_align[i])
    else:
        gpc_align_ow.append(gpc_align[i])

assert len(low_pid_ids.to_list()) == len(gpc_align_nw)
assert len(gpc_align_ow) + len(gpc_align_nw) == len(gpc_align)

### Remove all sites from alignment that introduce a gap in LASV Josiah GP

In [ ]:
keep_starts = []
keep_ends = []
drop_zero = False

drop_sites = []

for i in range(len(jos_full_align.seq)):
    if i != 0:
        if (jos_full_align[i] == '-') and (jos_full_align[i-1] != '-'):
            keep_ends.append(i)
        elif (jos_full_align[i] != '-') and (jos_full_align[i-1] == '-'):
            keep_starts.append(i)
    else:
        if jos_full_align[i] == '-':
            drop_zero = True
        else:
            keep_starts.append(i)
    
    if jos_full_align[i] == '-':
        drop_sites.append(i)
            

print(keep_starts)
print(keep_ends)
print(drop_zero)
# print()
# print(drop_sites)

## The below analyses are for all 3 GPC_all alignments (OW, NW, and all)

In [ ]:
gpc_align_ow_nogaps = gpc_align_ow[:, keep_starts[0]:keep_ends[0]]
gpc_align_nw_nogaps = gpc_align_nw[:, keep_starts[0]:keep_ends[0]]
gpc_align_all_nogaps = gpc_align[:, keep_starts[0]:keep_ends[0]]
for i in range(1, len(keep_starts)):
    gpc_align_ow_nogaps += gpc_align_ow[:, keep_starts[i]:keep_ends[i]]
    gpc_align_nw_nogaps += gpc_align_nw[:, keep_starts[i]:keep_ends[i]]
    gpc_align_all_nogaps += gpc_align[:, keep_starts[i]:keep_ends[i]]

print('OldWorld')
print(gpc_align_ow_nogaps)
print('\nNewWorld')
print(gpc_align_nw_nogaps)
print('\nAll')
print(gpc_align_all_nogaps)
print()

for seq in gpc_align_all_nogaps:
    if seq.description == jos_id:
        jos_full_align_nogaps = seq

print(jos_full_align_nogaps)
print()
print(jos_full_align_nogaps.seq)
print(len(jos_full_align_nogaps.seq))

### Process alignments

In [ ]:
print('OldWorld\n')
ow_align_df = process_alignment(jos_full_align_nogaps, gpc_align_ow_nogaps)
print('\nNewWorld\n')
nw_align_df = process_alignment(jos_full_align_nogaps, gpc_align_nw_nogaps)
print('\nAll\n')
all_align_df = process_alignment(jos_full_align_nogaps, gpc_align_all_nogaps)
all_align_df_min2 = process_alignment(jos_full_align_nogaps, gpc_align_all_nogaps, mincount=2)

## Number of mutations included in alignment

Add up `num_dif_nogap` plus the length of the alignment to determine single mutants in alignment.

In [ ]:
# exclude TM and cytoplasmic domains
align_df_dict = {'OldWorld': ow_align_df, 'NewWorld': nw_align_df, 'All': all_align_df, 'All_Min2': all_align_df_min2}
for subset in align_df_dict:
    print(f"\n{subset}")
    gp1_2 = align_df_dict[subset].iloc[:427]
    print('Muts in alignment excluding TM+C-tail:')
    print(gp1_2['num_dif_nogap'].sum() + len(gp1_2))

print('\nTotal muts in GPC (excluding TM+C-tail)')
print(427*20)

### Make heatmap of amino acid counts at each site for full alignment

In [ ]:
scc_heatmap = all_align_df.drop(['site', 'max_nogap', 'num_dif_nogap'], axis=1).transpose().replace(0, np.nan)
scc_hm_melt = scc_heatmap.melt(ignore_index=False).reset_index().rename(columns={'index': 'aa', 'variable': 'site', 'value': 'count'})

aa_order = ['R', 'K', 'H', 'D', 'E', 'Q', 'N', 'S', 'T',
            'Y', 'W', 'F', 'A', 'I', 'L', 'M', 'V', 'G', 'P', 'C', '-', 'X']
scc_hm_melt.aa = scc_hm_melt.aa.astype("category")
scc_hm_melt.aa.cat.set_categories(aa_order, inplace=True)

scc_hm_melt = scc_hm_melt.sort_values(['aa', 'site'])

display(scc_hm_melt.head())

In [ ]:
alt.data_transformers.disable_max_rows()

heatmap = alt.Chart(scc_hm_melt).mark_rect().encode(
                    alt.X('site:O'),
                    alt.Y("aa:O", sort=alt.EncodingSortField(field='order', order='ascending')),
                    color=alt.Color('count:Q', scale=alt.Scale(scheme="Blues")),
                    tooltip=[
                        alt.Tooltip('aa:O', title='Amino Acid'),
                        alt.Tooltip('site:O', title='Site'),
                        alt.Tooltip('count:Q', title='Count')
                        ]
                    ).properties(width=5000)

heatmap

### Drop amino acids with < 5 counts from heatmap

In [ ]:
scc_hm_melt_2 = scc_hm_melt.copy()
scc_hm_melt_2['count'] = scc_hm_melt_2['count'].replace([1, 2, 3, 4, 5], np.nan)
# scc_hm_melt_2['count'] = scc_hm_melt_2['count'].replace(2, np.nan)

heatmap = alt.Chart(scc_hm_melt_2).mark_rect().encode(
                    alt.X('site:O'),
                    alt.Y("aa:O", sort=alt.EncodingSortField(field='order', order='ascending')),
                    color=alt.Color('count:Q', scale=alt.Scale(scheme="Blues")),
                    tooltip=[
                        alt.Tooltip('aa:O', title='Amino Acid'),
                        alt.Tooltip('site:O', title='Site'),
                        alt.Tooltip('count:Q', title='Count')
                        ]
                    ).properties(width=5000)

heatmap

### Make line plots:

1. Max count for any one amino acid at each site
2. Number gaps at each site
3. Number different amino acids at each site

In [ ]:
for subset in align_df_dict:
    make_line_plots(align_df_dict[subset], subset)

In [ ]:
print(gpc_align[:, 85:105])

### Count sites that are conserved across each full GPC alignment in all sites upstream of TM+C-tail.

In [ ]:
for subset in align_df_dict:
    align_df = align_df_dict[subset]
    conserved_sites_subset = align_df[align_df['num_dif_nogap']==0].site
    conserved_sites_subset_list = []
    for site in conserved_sites_subset:
        if site <= 427:
            conserved_sites_subset_list.append(site)
    print(f"\n{subset}:")
    print(conserved_sites_subset_list)
    print(f"{len(conserved_sites_subset_list)} conserved sites in the ectodomain.")

# Conclusions

After discussing above results with Jesse and Tyler, I don't feel like we have sufficient information to necessarily want to limit our library based only on currently available sequences. 

Furthermore, titers of LASV Josiah pseudotyped-lentiviruses recovered from transduced cells are now very high (on the order of 1e5 TU/mL), meaning it is not as necessary to limit library size as it would have been with lower titers. 

Additionally, papers from non-human primate challenge studies have identified potential escape mutations in LASV GP Josiah following treatment with some of the antibodies I will test. 

Therefore, I will simply order NNS primers that span LASV GP Josiah. I will likely include the signal peptide in the library since the signal peptide is a part of the final GPC complex. However, I will likely exclude the transmembrane domain and cytoplasmic tail as mutations there are likely negative or neutral. Indeed, looking at the analyses above, the TM domain (immediately following site 427) is fairly conserved. There is more variability in the cytoplasmic tail, but that is likely not playing a large role in function/evolution/antigenicity.